# ANN Final Project: Poem Generation

Emma Sheridan and Jessica Petersen

In [228]:
import numpy as np
import pandas as pd 
import random
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
import requests as rq

In [229]:
df = pd.read_csv('poems.csv') # load dataset
df.head() # getting top 5 head

,author,content,poem name,age,type
0,WILLIAM SHAKESPEARE,Let the bird of loudest lay\r\nOn the sole Ara...,The Phoenix and the Turtle,Renaissance,Mythology & Folklore
1,DUCHESS OF NEWCASTLE MARGARET CAVENDISH,"Sir Charles into my chamber coming in,\r\nWhen...",An Epilogue to the Above,Renaissance,Mythology & Folklore
2,THOMAS BASTARD,"Our vice runs beyond all that old men saw,\r\n...","Book 7, Epigram 42",Renaissance,Mythology & Folklore
3,EDMUND SPENSER,"Lo I the man, whose Muse whilome did maske,\r\...","from The Faerie Queene: Book I, Canto I",Renaissance,Mythology & Folklore
4,RICHARD BARNFIELD,"Long have I longd to see my love againe,\r\nSt...",Sonnet 16,Renaissance,Mythology & Folklore


In [230]:
df.groupby('type').count()

,author,content,poem name,age
type,,,,
Love,326,326,326,326
Mythology & Folklore,59,59,58,59
Nature,188,188,187,188


In [231]:
# get the love, nature, and mythology poems 
poems = df['content']
lovePoems1 = poems.loc[df['type'] == 'Love']
mythPoems = poems.loc[df['type'] == 'Mythology & Folklore']
naturePoems = poems.loc[df['type'] == 'Nature']

In [232]:
lovePoems1 = lovePoems1.to_string()

In [233]:
# loading additional love poems from poetryfoundation.com dataset
df2 = pd.read_csv('PoetryFoundationData.csv') # load dataset
df2 = df2.dropna()
df2.head() # getting top 5 head

,Unnamed: 0,Title,Poem,Poet,Tags
6,6,\r\r\n Invisible Fish\r\r\n...,\r\r\nInvisible fish swim this ghost ocean now...,Joy Harjo,"Living,Time & Brevity,Relationships,Family & A..."
7,7,\r\r\n Don’t Bother the Ear...,\r\r\nDon’t bother the earth spirit who lives ...,Joy Harjo,"Religion,The Spiritual,Mythology & Folklore,Fa..."
9,9,"\r\r\n [""Hour in which I co...","\r\r\nHour in which I consider hydrangea, a sa...",Simone White,"Living,Parenthood,The Body,The Mind,Nature,Tre..."
16,16,\r\r\n scars\r\r\n ...,\r\r\nmy father’s body is a map\r\r\na record ...,Truong Tran,"The Body,Family & Ancestors"
17,17,\r\r\n what remains two\r\r...,\r\r\nit has long been forgotten this practice...,Truong Tran,"Infancy,Parenthood,The Body"


In [213]:
searchfor = ['Love', 'Relationships']

In [214]:
# getting additional poems that contain the tag Love 
poems2 = df2['Poem']
lovePoems2 = poems2.loc[df2['Tags'].str.contains('|'.join(searchfor))]

In [215]:
lovePoems2.head()
lovePoems2 = lovePoems2.to_string()

4707


In [234]:
# concatenate lovePoems and lovePoems 2 to create more data
lovePoems = lovePoems1 + lovePoems2

# remove numbers, ..., and \r\n, \r\r\n
lovePoems.replace('...', '')
lovePoems.replace('\\r\n', '')
lovePoems.replace('\\r\\n', '')
lovePoems.replace('...\n', '')
result = ''.join([i for i in lovePoems if not i.isdigit()])

In [226]:
result

'     Why didst thou promise such a beauteous day,\\r...\n     Weret aught to me I bore the canopy,\\r\\nWith m...\n     Let the bird of loudest lay\\r\\nOn the sole Ara...\n          Sing lullaby, as women do,\\r\\nWherewith t...\n     Beshrew that heart that makes my heart to groa...\n     O, call not me to justify the wrong\\r\\nThat th...\n     Love is my sin, and thy dear virtue hate,\\r\\nH...\n     My love is as a fever, longing still\\r\\nFor th...\n     A Poet am I neither born nor bred,\\r\\nBut to a...\n     Loving in truth, and fain in verse my love to ...\n     Misus and Mopsa hardly could agree,\\r\\nStrivin...\n     Joy of my life, full oft for loving you\\r\\n   ...\n     Sweet shades why doe you seeke to give delight...\n     Come darkest night, becoming sorrow best;\\r\\n ...\n     Love like a jugler, comes to play his prise,\\r...\n     Time only cause of my unrest\\r\\nBy whom I hopd...\n     Love peruse me, seeke, and finde\\r\\nHow each c...\n     When I beeheld t

In [172]:
mythPoems.head()

0    Let the bird of loudest lay\r\nOn the sole Ara...
1    Sir Charles into my chamber coming in,\r\nWhen...
2    Our vice runs beyond all that old men saw,\r\n...
3    Lo I the man, whose Muse whilome did maske,\r\...
4    Long have I longd to see my love againe,\r\nSt...
Name: content, dtype: object

In [49]:
naturePoems.head()

19    Why didst thou promise such a beauteous day,\r...
20    The welcome Sun from sea Freake is returned,\r...
21    I met a courtier riding on the plain,\r\nWell-...
22    Walking the fields a wantcatcher I spied,\r\nT...
23    Fishing, if I a fisher may protest,\r\nOf plea...
Name: content, dtype: object

In [174]:
# MAPPING CHARACTERS: for love poems, Make two dictionaries , one to convert chars to ints, the other to convert ints back to chars

chars = sorted(list(set(lovePoems)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [175]:
# process the dataset:
seqlen = 50
step = seqlen

data_X = []
data_y = []

poemLines = [] 

# creates poem lines 
for i in range(0, len(lovePoems) - seqlen - 1, step):
    poemLines.append(lovePoems[i: i + seqlen + 1])
    
# creating x and y data
data_X = np.zeros((len(poemLines), seqlen, len(chars)), dtype=np.bool)
data_Y = np.zeros((len(poemLines), seqlen, len(chars)), dtype=np.bool)

for i, poemLines in enumerate(poemLines):
    for t, (char_in, char_out) in enumerate(zip(poemLines[:-1], poemLines[1:])):
        data_X[i, t, char_indices[char_in]] = 1
        data_Y[i, t, char_indices[char_out]] = 1

In [176]:
# create the model
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=['categorical_crossentropy', 'accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 50, 128)           128000    
_________________________________________________________________
dense_7 (Dense)              (None, 50, 121)           15609     
Total params: 143,609
Trainable params: 143,609
Non-trainable params: 0
_________________________________________________________________


In [177]:
# train the model
model.fit(data_X, data_Y, batch_size=128, epochs=80)

Epoch 1/80
5925/5925 [==============================] - 10s 2ms/step - loss: 3.0610 - categorical_crossentropy: 3.0610 - accuracy: 0.2607
Epoch 2/80
5925/5925 [==============================] - 12s 2ms/step - loss: 2.0584 - categorical_crossentropy: 2.0584 - accuracy: 0.4366
Epoch 3/80
5925/5925 [==============================] - 11s 2ms/step - loss: 1.8034 - categorical_crossentropy: 1.8034 - accuracy: 0.4886
Epoch 4/80
5925/5925 [==============================] - 10s 2ms/step - loss: 1.6841 - categorical_crossentropy: 1.6841 - accuracy: 0.5159
Epoch 5/80
5925/5925 [==============================] - 10s 2ms/step - loss: 1.5964 - categorical_crossentropy: 1.5964 - accuracy: 0.5366
Epoch 6/80
5925/5925 [==============================] - 10s 2ms/step - loss: 1.5448 - categorical_crossentropy: 1.5448 - accuracy: 0.5472
Epoch 7/80
5925/5925 [==============================] - 11s 2ms/step - loss: 1.5026 - categorical_crossentropy: 1.5026 - accuracy: 0.5571
Epoch 8/80
5925/5925 [============

5925/5925 [==============================] - 10s 2ms/step - loss: 1.0775 - categorical_crossentropy: 1.0775 - accuracy: 0.6683
Epoch 61/80
5925/5925 [==============================] - 10s 2ms/step - loss: 1.0808 - categorical_crossentropy: 1.0808 - accuracy: 0.6668
Epoch 62/80
5925/5925 [==============================] - 10s 2ms/step - loss: 1.0781 - categorical_crossentropy: 1.0781 - accuracy: 0.6678
Epoch 63/80
5925/5925 [==============================] - 10s 2ms/step - loss: 1.0781 - categorical_crossentropy: 1.0781 - accuracy: 0.6682
Epoch 64/80
5925/5925 [==============================] - 10s 2ms/step - loss: 1.0748 - categorical_crossentropy: 1.0748 - accuracy: 0.6692
Epoch 65/80
5925/5925 [==============================] - 10s 2ms/step - loss: 1.0736 - categorical_crossentropy: 1.0736 - accuracy: 0.6696
Epoch 66/80
5925/5925 [==============================] - 10s 2ms/step - loss: 1.0725 - categorical_crossentropy: 1.0725 - accuracy: 0.6696
Epoch 67/80
5925/5925 [================

In [179]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)  

In [180]:
def generating_poem(epoch, start_index, diversity, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = start_index
    diversity = diversity

    generated = ''
    poemLines = lovePoems[start_index: start_index + seqlen]
    generated += poemLines
    print('----- Generating with seed: "' + poemLines + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(poemLines):
            x_pred[0, t, char_indices[char]] = 1.
            
        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        poemLines = poemLines[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [181]:
generating_poem(80, 50, .25, _)


----- Generating text after Epoch: 80
----- Generating with seed: ",\r...
73     Weret aught to me I bore the canopy,"
,\r...
73     Weret aught to me I bore the canopy,\r\r\n\r\r\nthe...
298      \r\r\n\r\r\n\r\r\n\r\r\nFor Mac)\r\r\n\r\r\...
295      \r\r\nThe sun the sun the street the sun the stril...
5915     \r\r\nWhen I can there is a stanst that that we calle...
12860    \r\r\n\r\r\n\r\r\n\r\r\nThere like themed my love there is the strange song...
10990    \r\r\nThere is no delught of the street be of the st...
11942    \r\r\nAnd there is a stands of Lov
